In [3]:
import json
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow import keras

# Path to the dataset file
DATASET_PATH = "data.json"

def load_data(dataset_path):
    """
    Loads the dataset from a JSON file and returns the inputs and targets as numpy arrays.

    Parameters:
    dataset_path (str): Path to the dataset file.

    Returns:
    inputs (np.array): The MFCCs extracted from the audio files.
    targets (np.array): The corresponding labels (genres) for the audio files.
    """
    with open(dataset_path, "r") as fp:
        data = json.load(fp)
    inputs = np.array(data["mfcc"])
    targets = np.array(data["labels"])

    return inputs, targets

if __name__ == "__main__":
    # Load the dataset
    inputs, targets = load_data(DATASET_PATH)

    # Split data into training and testing sets (70% training, 30% testing)
    inputs_train, inputs_test, targets_train, targets_test = \
        train_test_split(inputs, targets, test_size=0.3)

    # Build the neural network architecture
    model = keras.Sequential([
        # Input layer: Flatten the input to be fed into the dense layers
        keras.layers.Flatten(input_shape=(inputs.shape[1], inputs.shape[2])),
        # 1st hidden layer with 512 neurons and ReLU activation function
        keras.layers.Dense(512, activation="relu"),
        # 2nd hidden layer with 256 neurons and ReLU activation function
        keras.layers.Dense(256, activation="relu"),
        # 3rd hidden layer with 64 neurons and ReLU activation function
        keras.layers.Dense(64, activation="relu"),
        # Output layer with 10 neurons (one for each genre) using softmax activation
        keras.layers.Dense(10, activation="softmax")
    ])

    # Compile the model using Adam optimizer and a learning rate of 0.0001
    optimizer = keras.optimizers.Adam(learning_rate=0.0001)
    model.compile(
        optimizer=optimizer,
        loss="sparse_categorical_crossentropy", # Suitable loss function for classification
        metrics=["accuracy"] # Track accuracy during training
    )

    # Print the summary of the model architecture
    model.summary()

    # Train the model using the training data
    model.fit(
        inputs_train,
        targets_train,
        validation_data=(inputs_test, targets_test), # Validate on the test set
        epochs=50, # Train for 50 epochs
        batch_size=32 # Use a batch size of 32
    )


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 1690)              0         
                                                                 
 dense_1 (Dense)             (None, 512)               865792    
                                                                 
 dense_2 (Dense)             (None, 256)               131328    
                                                                 
 dense_3 (Dense)             (None, 64)                16448     
                                                                 
 dense_4 (Dense)             (None, 10)                650       
                                                                 
Total params: 1014218 (3.87 MB)
Trainable params: 1014218 (3.87 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/50
197/1